In [ ]:
import numpy as np
import datetime
import shapefile
import xarray as xr

from tools.utils import create_polydict_ptsarr, xy2ll, ll2xy
from tools.landsat_utils import easy_raster, normalize_band as normalize
from tools.itslive_utils import flowline_sample, DATACUBETOOLS, running_mean

import matplotlib.pyplot as plt
from tools.plot_tools import plot_latlon_lines
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib import gridspec

In [ ]:
import plotSettings
plotSettings.paper()
COLS = plotSettings.COLS

In [ ]:
%matplotlib tk

In [ ]:
# general data directory
data_dir = '/'

In [ ]:
center_lon, center_lat = 69.5881, -72.0370
center_x, center_y = ll2xy(center_lon, center_lat)

# flowlines shape file for both ITS_LIVE sampling and plotting
# include grounded ice
ITS_LIVE_flowlines = 'GIS/Flowlines.shp'
fl_shps = shapefile.Reader(os.path.join(data_dir, ITS_LIVE_flowlines)).shapes()

# an image to show where the flow lines are
flowline_img_files_dir = 'data/LandSat8-9_Color/LC09_L1GT_126111_20230129_20230311_02_T2'

# bounding boxes for masking images 
bounding_box_file = 'GIS/BoundingBoxes2.shp'
bbs_dict, _ = create_polydict_ptsarr(os.path.join(data_dir, bounding_box_file))
bb = bbs_dict[0]

print('loading flowline velocities')
dss = flowline_sample(os.path.join(data_dir, ITS_LIVE_flowlines), Npts=30, epsg="3031")

In [ ]:
SAVE = False
save_name = 'figure5'
save_dir = 'figs'
save_type = '.svg'

############################################################################################################
# plot settings
############################################################################################################
collapse_timerange = [datetime.datetime(2015,3,29), datetime.datetime(2015,4,3)]
timeseries_range = [datetime.datetime(2013,12,6),datetime.datetime(2017,4,3)]
timeseries_range = xr.date_range(timeseries_range[0], timeseries_range[-1])

grounding_line_opts = {'color': 'k', 'lw': 0.5, 'ls':'-'}

linestyles = [':', (0, (3, 1, 1, 1, 1, 1)), '--', '-']
markers=['o', 'x', '+', 'd']

############################################################################################################
# create figure and all subplots
############################################################################################################
plt.close('all')
fig = plt.figure(num=2,clear=1, figsize=[13.13066667,  1.1*5.056])

PARENT = gridspec.GridSpec(2,4, figure=fig,
                           width_ratios=[1,1,1.2,1.2])
gs0 = gridspec.GridSpecFromSubplotSpec(1,1,subplot_spec=PARENT[:,:2])
imax = fig.add_subplot(gs0[0])

gs3 = gridspec.GridSpecFromSubplotSpec(2,1, subplot_spec=PARENT[:,2:], wspace=0.3,
                                       hspace=0.015)
ax = [fig.add_subplot(gs3[0]), fig.add_subplot(gs3[1])]

############################################################################################################
# plot context image
############################################################################################################

rast = easy_raster(os.path.join(data_dir, flowline_img_files_dir), mask=bbs_dict[1], always_TOA=True)
left, bottom, right, top = bbs_dict[1].bounds
R,G,B = rast.red, rast.green, rast.blue
MIN = (0,0,0)
MAX = [np.quantile(b, 0.98) for b in [R,G,B]]
shape = R.shape
RGB = np.empty((shape[0], shape[1], 3),dtype=int)
RGB[:,:,0] = 255*(normalize(R, MIN[0], MAX[0]) + 0.)
RGB[:,:,1] = 255*(normalize(G, MIN[1], MAX[1]) + 0.)
RGB[:,:,2] = 255*(normalize(B, MIN[2], MAX[2]) + 0.)
    
imax.imshow(RGB, extent=(left, right, bottom, top), rasterized=True, aspect='equal')
imax.set(xticklabels=[], yticklabels=[],
            xticks=[], yticks=[],
            xlim=(left, right), ylim=(bottom, top),
            )
ll=xy2ll(left, bottom), xy2ll(left, top), xy2ll(right, top), xy2ll(right, bottom)
ll=np.array(ll)
lon_range = [ll[:,0].min(), ll[:,0].max()]
lat_range = [ll[:,1].min(), ll[:,1].max()]
imax = plot_latlon_lines(imax, 
                            lat_range=lat_range, lon_range=lon_range,
                            lat_step=0.5, lon_step=1,
                            extents=(left, right, bottom, top),
                           )

for i,fl in enumerate(fl_shps):
    pts = np.array(fl.points)
    imax.plot(pts[:,0], pts[:,1], 
                 ls=linestyles[i], c='k')

imax.plot(bbs_dict[4].exterior.xy[0], bbs_dict[4].exterior.xy[1], **grounding_line_opts)
for intr in bbs_dict[4].interiors:
    imax.plot(intr.xy[0], intr.xy[1], **grounding_line_opts)
    
imax.scatter(center_x, center_y, c=COLS[0], marker='d', zorder=99)

[i.set_linewidth(1) for _,i in imax.spines.items()]

scalebar = AnchoredSizeBar(imax.transData,
                           10e3, '10 km', 'lower right', 
                           pad=1,
                           frameon=False,
                           size_vertical=500,
                          )
[a.set_linewidth(1) for _,a in imax.spines.items()]
imax.add_artist(scalebar)

############################################################################################################
# plot itslive flowline speed
############################################################################################################
print('plotting flowline speeds')
ax2 = ax[0].twinx()
ax2.set_ylabel('Standardized Anomaly', color=COLS[1])

for i,ds in enumerate(dss):
    ds_m = ds.resample(mid_date='1M').mean()
    ax[0].plot(ds_m.sel(mid_date=timeseries_range,method='nearest').mid_date, 
               ds_m.mean(dim='x').sel(mid_date=timeseries_range,method='nearest').v, 
               ls=linestyles[i], c=COLS[0])
    # standardized anomaly
    SA=xr.apply_ufunc(
        lambda x,m,s: (x-m)/s,
        ds.groupby('mid_date.month'),
        ds.groupby('mid_date.month').mean(),
        ds.groupby('mid_date.month').std(),
        dask='parallelized'
    )
    ds_std = SA.resample(mid_date='1M').std()
    ds_m = SA.resample(mid_date='1M').mean()
    
    ax2.plot(ds_m.mid_date.sel(mid_date=timeseries_range,method='nearest').values, 
             ds_m.mean(dim='x').sel(mid_date=timeseries_range,method='nearest').v,
             ls=linestyles[i], c=COLS[1],
            )#marker=markers[i], ms=5)
    

    
ax[0].axvspan(collapse_timerange[0],collapse_timerange[-1], 
                fc='k', ec='none',alpha=0.33)
ax[0].set_ylabel('Ice surface speed (m/yr)', color=COLS[0])
ax[0].set(xlim=(timeseries_range[0], timeseries_range[-1]),
          xticklabels=[])
          
ax[0].tick_params(axis='y', labelcolor=COLS[0])

ax2.axhline(0, c='k', ls='--')
ax2.tick_params(axis='y', labelcolor=COLS[1])
ax2.set_ylabel('Standardized Anomaly', color=COLS[1])

[a.spines[loc].set_visible(False) for a,loc in zip((ax[0], ax2), ('bottom','bottom'))]
[a.set_xticks([]) for a in (ax[0], ax2)]

############################################################################################################
# plot itslive point speed
############################################################################################################
print('plotting point speed')
DT = DATACUBETOOLS()
_,ds,_ = DT.get_timeseries_at_point((center_x, center_y), '3031', variables=['v'])
rsm,tsm = running_mean(ds.mid_date.values, ds.v.values, 5, 30)
ax[1].scatter(ds.mid_date, ds.v,
              marker='d', fc='k', ec='none', alpha=0.125)
ax[1].plot(tsm, rsm, marker='d', c=COLS[0])
ax[1].set_ylim(np.nanmin(rsm) * 0.94, 
               np.nanmax(rsm) * 1.05)

ds = xr.DataArray(data=rsm[:,0], dims=['mid_date'], coords=dict(mid_date=tsm))
SA=xr.apply_ufunc(
    lambda x,m,s: (x-m)/s,
    ds.groupby('mid_date.month'),
    ds.groupby('mid_date.month').mean(),
    ds.groupby('mid_date.month').std(),
    dask='parallelized'
)
ax2 = ax[1].twinx()
ax2.plot(SA.mid_date.sel(mid_date=timeseries_range,method='nearest').values, 
         SA.sel(mid_date=timeseries_range,method='nearest'),
         c=COLS[1], marker='d')


ax[1].axvspan(collapse_timerange[0],collapse_timerange[-1], 
                fc='k', ec='none',alpha=0.33)
ax[1].set_ylabel('Ice surface speed (m/yr)', color=COLS[0])
ax[1].set(xlim=(timeseries_range[0], timeseries_range[-1]),)
ax[1].tick_params(axis='y', labelcolor=COLS[0])
ax[1].tick_params(axis='x', rotation=45)

ax2.axhline(0, c='k', ls='--')
ax2.tick_params(axis='y', labelcolor=COLS[1])
ax2.set_ylabel('Standardized Anomaly', color=COLS[1])

[a.spines[loc].set_visible(False) for a,loc in zip((ax[1], ax2), ('top', 'top'))]
[a.tick_params(axis='x', which='both', bottom=True, top=False,) for a in [ax[1], ax2]]

if SAVE:
    plt.savefig(os.path.join(save_dir, save_name + fig_type), dpi=600.)